In [ ]:
import nltk

In [ ]:
import pickle

In [ ]:
import random

In [ ]:
import numpy as np

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD

In [ ]:
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
from nltk.corpus import stopwords

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
ignore_words = stopwords.words('english')

In [ ]:
ignore_words

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [ ]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [ ]:
import json

In [ ]:
data_file=open(r'dashboard.json').read()

In [ ]:
intents=json.loads(data_file)

In [ ]:
intents

{'intents': [{'context': [''],
   'patterns': ['open dashboard',
    'show dashboard',
    'display dashboard',
    'enter dashboard',
    'prepare dashboard'],
   'responses': ['Dynamic Dashboard Opened in a new Window'],
   'tag': 'Open_Dashboard'},
  {'context': [''],
   'patterns': ['what is the shape of dataset',
    'what is size of dataset',
    'lenght of dataset'],
   'responses': ['Here is the size of dataset:'],
   'tag': 'shape'},
  {'context': [''],
   'patterns': ['what are the columns of dataset',
    'show the dataset attributes',
    'fields in dataset',
    'give the columns of dataset'],
   'responses': ['Here are the columns of the dataset:'],
   'tag': 'columns'},
  {'context': [''],
   'patterns': ['plot line chart',
    'show line graph',
    'draw line graph',
    'give line plot'],
   'responses': ['Line plot opened in a new window:'],
   'tag': 'lineplot'},
  {'context': [''],
   'patterns': ['what is this dataset about',
    'tell me something about the datas

In [ ]:
words=[]
documents=[]
classes=[]
for intent in intents['intents']:
  for pattern in intent['patterns']:
    w=nltk.word_tokenize(pattern)
    words.extend(w)
    documents.append((w,intent['tag']))
    if intent['tag'] not in classes:
      classes.append(intent['tag'])

In [ ]:
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

In [ ]:
classes = sorted(list(set(classes)))

In [ ]:
print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique lemmatized words", words)

85 documents
19 classes ['Open_Dashboard', 'Top_dealers', 'address', 'age', 'barplot', 'columns', 'datadir', 'dataset', 'goodbye', 'great', 'greeting', 'lineplot', 'name', 'options', 'selectdataset', 'shape', 'summary', 'thanks', 'wellbeing']
94 unique lemmatized words ["'s", ',', ':', '?', '\\data', 'ad-hocbot', 'address', 'age', 'any', 'anyone', 'attribute', 'awesome', 'bar', 'bye', 'call', 'chart', 'chatting', 'choose', 'column', 'could', 'dashboard', 'dashboards\\consolidated', 'data', 'data.xlsx', 'dataset', 'day', 'dealer', 'describe', 'description', 'detail', 'display', 'draw', 'enter', 'f', 'field', 'find', 'fine', 'give', 'good', 'goodbye', 'graph', 'great', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'hola', 'how', 'i', 'information', 'input', 'is', 'later', 'lenght', 'line', 'live', 'load', 'location', 'name', 'next', 'nice', 'offered', 'old', 'open', 'plot', 'prepare', 'provide', 'retail', 'see', 'select', 'sets\\iris-species\\iris.xlsx', 'sets\\ttl', 'shape', 'show

In [ ]:
pickle.dump(words,open(r'words.pkl','wb'))

In [ ]:
pickle.dump(classes,open(r'classes.pkl','wb'))

In [ ]:
training = []
output_empty = [0] * len(classes)
for doc in documents:
  bag = []
  pattern_words = doc[0]
  pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
  for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
  output_row = list(output_empty)
  output_row[classes.index(doc[1])] = 1
  training.append([bag, output_row])

In [ ]:
random.shuffle(training)
training = np.array(training)

In [ ]:
train_x = list(training[:,0])
train_y = list(training[:,1])

In [ ]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

In [ ]:
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [ ]:
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)

Epoch 1/200
17/17 [==============================] - 0s 2ms/step - loss: 2.9621 - accuracy: 0.0235
Epoch 2/200
17/17 [==============================] - 0s 1ms/step - loss: 2.9356 - accuracy: 0.0353
Epoch 3/200
17/17 [==============================] - 0s 1ms/step - loss: 2.8368 - accuracy: 0.1294
Epoch 4/200
17/17 [==============================] - 0s 1ms/step - loss: 2.8504 - accuracy: 0.1176
Epoch 5/200
17/17 [==============================] - 0s 1ms/step - loss: 2.7362 - accuracy: 0.2471
Epoch 6/200
17/17 [==============================] - 0s 1ms/step - loss: 2.6780 - accuracy: 0.2353
Epoch 7/200
17/17 [==============================] - 0s 1ms/step - loss: 2.5745 - accuracy: 0.2941
Epoch 8/200
17/17 [==============================] - 0s 1ms/step - loss: 2.4372 - accuracy: 0.2824
Epoch 9/200
17/17 [==============================] - 0s 2ms/step - loss: 2.3897 - accuracy: 0.3176
Epoch 10/200
17/17 [==============================] - 0s 2ms/step - loss: 2.3485 - accuracy: 0.3529
Epoch 11/

In [ ]:
model.save(r'chatbot_model.h5', hist)